In [1]:
import csv

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

with open('label_data.csv', encoding='utf8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='~')
    line_count = 0
    data = []
    for row in csv_reader:
        row[1] = deEmojify(row[1])
        data.append(row)
        line_count+=1
    print(f'Processed {line_count} lines.')

Processed 673 lines.


In [2]:
good = 0
bad = 0

for x, k in enumerate(data):
    if x == 0:
        continue
    else:
        if len(k) != 4:
            print(f"Error: {x,k}")
            break
        elif k[1] == "1":
            good += 1
        elif k[1] == "0":
            bad += 1
        else:
            print(f"MG: {x,k}")

print(good, bad)

336 336


In [3]:
# PREPPING DATA

import json

copy_of_data = data.copy()

In [4]:
copy_of_data.pop(0)

for d in copy_of_data:
    d.remove(d[0])

In [5]:
training_data = []
for ddd in copy_of_data:
    appe = f"""{ddd[1]} | {ddd[2]}  ->"""
    training_data.append(
        {
            "prompt": appe,
            "completion": str(bool(int(ddd[0]))) + "."
        }
    )

In [6]:
x1 = []
x0 = []

for x in training_data:
    if x["completion"] == "False.":
        x0.append(x)
    else:
        x1.append(x)

In [7]:
print(len(x1), len(x0))

# split dataset 80:20
x1_train = x1[:int(len(x1)*0.8)]
x1_test = x1[int(len(x1)*0.8):]


x0_train = x0[:int(len(x0)*0.8)]
x0_test = x0[int(len(x0)*0.8):]

train = x1_train + x0_train
val = x1_test + x0_test

336 336


In [8]:
print("Validation: ", len(val))
print("Train: ", len(train))

Validation:  136
Train:  536


In [15]:
import random
random.shuffle(val)

In [9]:
val[1]

{'prompt': "How to make a faceless youtube channel. #ai #aitools  | How to make a faceless YouTube money printer with AI? This method is so easy my left nut could do it. Step 1. I want you to go to vidIQ.com and let it tell you what are the best keywords and best topics for you to create your YouTube channel on. Step 2. You're going to go to ChatGPT and literally just give it a command that says, Hey, ChatGPT, can you give me 50 video ideas about this topic? Step 3. Go to Speechify.com, take that script, upload it. It'll create an actual voice reading that script. Step 4. Go to Video.io, take that script and that voice, upload it there. It'll create an entire video using amazing stock footage. Step 5. Canva.com, create yourself an amazing YouTube thumbnail. And then last step, go back to ChatGPT, say, Hey, write me a video title and a YouTube description for this video. Copy, paste, upload to YouTube. Do this once or twice a day consistently. You're going to make decent money off ad re

In [35]:
with open("training_data.jsonl", "w", encoding="UTF-8") as file:
    for x in train:
        file.write(json.dumps(x) + "\n")

In [ ]:
# This is the process I used for fine-tuning: https://platform.openai.com/docs/guides/fine-tuning

# Use they're data preperation tool
# Fine-Tune on Specified Model

In [30]:
import openai
response = openai.Completion.create(
      model="ada:ft-personal-2023-07-24-17-57-29",
      prompt=deEmojify(val[2]["prompt"]),
      temperature=1,
      top_p=1,
      logprobs=2, # see probabilities of all of em
      frequency_penalty=0,
      presence_penalty=0,
      stop=["."]
    )
print(response)

print(val[2]["completion"])

# log probabilities to probabilities
import math

def logprobs_to_probs(logprobs):
    probs = []
    for x in logprobs:
        probs.append(math.exp(x))
    return probs



{
  "id": "cmpl-7gfpo3ujBcTDj9AfTS37qHrxlrBXx",
  "object": "text_completion",
  "created": 1690404944,
  "model": "ada:ft-personal-2023-07-24-17-57-29",
  "choices": [
    {
      "text": " True",
      "index": 0,
      "logprobs": {
        "tokens": [
          " True"
        ],
        "token_logprobs": [
          -0.009088164
        ],
        "top_logprobs": [
          {
            " False": -4.9305844,
            " True": -0.009088164
          }
        ],
        "text_offset": [
          858
        ]
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 198,
    "completion_tokens": 1,
    "total_tokens": 199
  }
}
True.


In [31]:

print(
    logprobs_to_probs(response["choices"][0]["logprobs"]["top_logprobs"][0].values())
    )

[0.007222281346628315, 0.9909530085404407]


In [32]:
import openai

correct = 0
wrong = 0
c = 0

for x in val:
    response = openai.Completion.create(
      model="ada:ft-personal-2023-07-24-17-57-29",
      prompt=deEmojify(x["prompt"]),
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["."]
    )
    c+=1
    print(c, len(val))
    is_true = "true" in response["choices"][0]["text"].lower()
    label_is_true = "true" in x["completion"].lower()

    if is_true == label_is_true:
      correct += 1
    else:
      wrong += 1


1 136
2 136
3 136
4 136
5 136
6 136
7 136
8 136
9 136
10 136
11 136
12 136
13 136
14 136
15 136
16 136
17 136
18 136
19 136
20 136
21 136
22 136
23 136
24 136
25 136
26 136
27 136
28 136
29 136
30 136
31 136
32 136
33 136
34 136
35 136
36 136
37 136
38 136
39 136
40 136
41 136
42 136
43 136
44 136
45 136
46 136
47 136
48 136
49 136
50 136
51 136
52 136
53 136
54 136
55 136
56 136
57 136
58 136
59 136
60 136
61 136
62 136
63 136
64 136
65 136
66 136
67 136
68 136
69 136
70 136
71 136
72 136
73 136
74 136
75 136
76 136
77 136
78 136
79 136
80 136
81 136
82 136
83 136
84 136
85 136
86 136
87 136
88 136
89 136
90 136
91 136
92 136
93 136
94 136
95 136
96 136
97 136
98 136
99 136
100 136
101 136
102 136
103 136
104 136
105 136
106 136
107 136
108 136
109 136
110 136
111 136
112 136
113 136
114 136
115 136
116 136
117 136
118 136
119 136
120 136
121 136
122 136
123 136
124 136
125 136
126 136
127 136
128 136
129 136
130 136
131 136
132 136
133 136
134 136
135 136
136 136


In [32]:
# Curie
print(correct, wrong)

124 12


In [31]:
print(correct/(correct+wrong))

0.9117647058823529


In [33]:
# Ada
print(correct, wrong)

118 18


In [34]:
print(correct/(correct+wrong))

0.8676470588235294
